<a href="https://colab.research.google.com/github/efkomarova/Sirius/blob/main/Creational_Patterns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Порождающие паттерны (Creational)

часть 1: **как создавать объекты**.

- **Singleton** — один объект на всё приложение
- **Factory Method** — один метод, разный тип по параметру
- **Abstract Factory** — набор связанных объектов
- **Builder** — сборка по шагам
- **Prototype** — clone() вместо создания с нуля

Для запуска C++ в Colab: каждая ячейка записывает `.cpp` файл, компилирует и запускает его.

## 1. Singleton — один на всех

**Легенда:** Магнитик на холодильник — один на всех в семье. Не у каждого свой.

**Зачем нужен:** Иногда нужен ровно один объект на всё приложение: настройки, логгер, подключение к базе. Singleton гарантирует: сколько бы раз ни попросили — получишь тот же объект.

**Как работает:**
- Конструктор **приватный** — снаружи нельзя `new FridgeMagnet()`
- **Статическая** переменная `instance` — одна на весь класс
- Единственный способ получить объект — `FridgeMagnet::get()`

**Ключ в коде:** `static` переменная + приватный конструктор.

In [1]:
%%writefile singleton.cpp
/*
  ЛЕГЕНДА: Один общий магнитик на холодильник — все к нему.
  СУТЬ: Гарантирует один экземпляр на всё приложение.
  КОГДА: конфиг, логгер, подключение к БД.
*/

#include <iostream>
using namespace std;

// Класс — «чертёж» объекта. Singleton = один экземпляр на всю программу.
class FridgeMagnet {
    // static = одна переменная на ВСЕ объекты (общая для класса)
    static FridgeMagnet* instance;
    // Приватный конструктор! Снаружи нельзя: new FridgeMagnet()
    // Это ключ Singleton — создание только через get()
    FridgeMagnet() {}

public:
    // static — вызывается через класс: FridgeMagnet::get(), не через объект
    static FridgeMagnet* get() {
        if (!instance)                    // первый вызов — создаём
            instance = new FridgeMagnet();
        return instance;                  // всегда возвращаем тот же объект
    }
    void pin(const string& s) { cout << s << endl; }
};

// Статическую переменную ОБЯЗАТЕЛЬНО инициализировать вне класса
FridgeMagnet* FridgeMagnet::instance = nullptr;  // изначально «нет объекта»

int main() {
    // Два вызова get() — получаем ОДИН И ТОТ ЖЕ объект
    FridgeMagnet* m1 = FridgeMagnet::get();
    FridgeMagnet* m2 = FridgeMagnet::get();
    cout << (m1 == m2 ? "Один магнитик" : "Разные") << endl;  // m1 == m2!
    m1->pin("Напоминание");  // -> = обращение через указатель (как . для обычной переменной)
}

Writing singleton.cpp


In [2]:
!g++ -std=c++17 -o out singleton.cpp && ./out

Один магнитик
Напоминание


## 2. Factory Method — одна кнопка, разный результат

**Легенда:** Автомат с напитками. Нажал «чай» — получил чай. «Кофе» — кофе. Одна кнопка, разный результат.

**Зачем нужен:** Часто не знаешь заранее, объект какого класса понадобится. Одна функция `press(кнопка)` решает — вся логика «что создавать» в одном месте.

**Как работает:**
- Базовый класс `Drink` с методом `pour()`
- Наследники: `Tea`, `Coffee`
- Функция `press(кнопка)` по параметру создаёт нужный объект
- Возвращает указатель на базовый класс — клиент не знает конкретный тип

In [3]:
%%writefile factory_method.cpp
/*
  ЛЕГЕНДА: Автомат — нажал «чай», получил чай. Одна кнопка, разный результат.
  СУТЬ: Один метод создания, тип определяет результат.
*/

#include <iostream>
#include <memory>  // unique_ptr — умный указатель, сам освободит память
using namespace std;

// Базовый класс — абстрактный (есть метод = 0 — без реализации)
// virtual = можно переопределить в наследнике; полиморфизм
class Drink {
public:
    virtual void pour() = 0;   // = 0 = чисто виртуальный, класс нельзя создать
    virtual ~Drink() = default; // виртуальный деструктор нужен при delete через Drink*
};

// Tea наследует Drink: class Tea : public Drink
class Tea : public Drink {
    void pour() override { cout << "Чай налит\n"; }  // override — переопределяем
};
class Coffee : public Drink {
    void pour() override { cout << "Кофе налит\n"; }
};

// ФАБРИЧНЫЙ МЕТОД: одна функция по параметру решает, какой объект создать
// Возвращаем unique_ptr<Drink> — клиенту не важно, Tea или Coffee
unique_ptr<Drink> press(const string& btn) {
    if (btn == "tea")   return make_unique<Tea>();
    if (btn == "coffee") return make_unique<Coffee>();
    return nullptr;  // неизвестная кнопка
}

int main() {
    auto d1 = press("tea");    // auto — компилятор выведет тип (unique_ptr<Drink>)
    auto d2 = press("coffee");
    d1->pour();  // полиморфизм: вызовется Tea::pour()
    d2->pour();  // здесь Coffee::pour()
}

Writing factory_method.cpp


In [4]:
!g++ -std=c++17 -o out factory_method.cpp && ./out

Чай налит
Кофе налит


## 3. Abstract Factory — набор вместе

**Легенда:** Меню в кафе. «Завтрак» — каша, булка, кофе. «Обед» — суп, мясо, компот. Всё в наборе сочетается.

**Зачем нужен:** Когда нужно создать несколько связанных объектов, которые должны подходить друг к другу. Тёмная тема — все кнопки тёмные. Светлая — все светлые.

**Отличие от Factory Method:** Factory Method создаёт один объект. Abstract Factory — целое семейство, связанных общим «стилем».

In [5]:
%%writefile abstract_factory.cpp
/*
  ЛЕГЕНДА: Меню — завтрак (каша+булка) или обед (суп+мясо). Набор вместе.
  СУТЬ: Семейство связанных объектов — все из одного «набора» сочетаются.
*/

#include <iostream>
using namespace std;

// Абстрактный продукт — блюдо. Нельзя создать Dish напрямую.
class Dish {
public:
    virtual void serve() = 0;  // каждый наследник реализует по-своему
    virtual ~Dish() = default;
};

// Конкретные блюда — из «меню завтрак» и «меню обед»
class BreakfastEgg : public Dish {
    void serve() override { cout << "Яичница\n"; }
};
class DinnerSteak : public Dish {
    void serve() override { cout << "Стейк\n"; }
};

// Фабрика: по типу меню создаёт блюдо из нужного набора
// В реальности было бы: createFirstCourse(), createSecondCourse() — все из одного стиля
Dish* createDish(const string& meal) {
    if (meal == "breakfast") return new BreakfastEgg();
    return new DinnerSteak();
}

int main() {
    // Клиент не знает конкретные классы — работает через Dish*
    Dish* d1 = createDish("breakfast");
    Dish* d2 = createDish("dinner");
    d1->serve();
    d2->serve();
    delete d1; delete d2;  // с new обязательно delete!
}

Writing abstract_factory.cpp


In [6]:
!g++ -std=c++17 -o out abstract_factory.cpp && ./out

Яичница
Стейк


## 4. Builder — по шагам

**Легенда:** Бутерброд. Сначала хлеб, потом колбаса, потом сыр. Собираешь по слоям, шаг за шагом.

**Зачем нужен:** Когда у объекта много частей и важен порядок. Constructor с 10 параметрами — неудобно. Builder даёт вызывать `add()` по одному.

**Ключ в коде:** Объект с методом `add()`, который накапливает данные. Финальный метод `build()` или `show()`.

In [7]:
%%writefile builder.cpp
/*
  ЛЕГЕНДА: Бутерброд — хлеб, потом колбаса, потом сыр. По слоям.
  СУТЬ: Собирает сложный объект по шагам. Порядок вызовов важен!
*/

#include <iostream>
#include <string>
using namespace std;

// Объект накапливает части. Вместо конструктора с 10 параметрами — add() по одному
class Sandwich {
public:
    string layers;  // накапливаем слои в строке
    void add(const string& s) { layers += s + " "; }  // += добавляет к строке
    void show() { cout << "Бутерброд: " << layers << endl; }  // вывод готового
};

// Функция «сборки» — по шагам. В паттерне Builder это мог бы быть отдельный класс
Sandwich make() {
    Sandwich s;
    s.add("хлеб");    // шаг 1 — основа
    s.add("колбаса"); // шаг 2
    s.add("сыр");     // шаг 3
    return s;         // возвращаем готовый объект (копируется)
}

int main() {
    Sandwich sw = make();
    sw.show();
}

Writing builder.cpp


In [8]:
!g++ -std=c++17 -o out builder.cpp && ./out

Бутерброд: хлеб колбаса сыр 


## 5. Prototype — копия вместо нового

**Легенда:** Ключ от квартиры. Потерял — сделали дубликат с твоего старого. Не куют с нуля.

**Зачем нужен:** Создание с нуля бывает дорогим. Если нужен похожий объект — проще скопировать готовый и чуть изменить.

**Как работает:** Метод `clone()` создаёт новый объект того же типа, копирует поля, возвращает. Оригинал не меняется.

In [9]:
%%writefile prototype.cpp
/*
  ЛЕГЕНДА: Ключ — сделали дубликат. Не новый, копия.
  СУТЬ: Создание через clone() вместо expensive создания с нуля.
*/

#include <iostream>
#include <memory>
using namespace std;

// Базовый класс с методом клонирования. clone() = «сделай копию себя»
class Key {
public:
    string id;
    virtual unique_ptr<Key> clone() = 0;  // абстрактный — наследники реализуют
    virtual ~Key() = default;
};

// DoorKey умеет создавать свою копию — не через конструктор «с нуля»
class DoorKey : public Key {
public:
    unique_ptr<Key> clone() override {
        auto k = make_unique<DoorKey>();  // новый объект того же типа
        k->id = id;                       // копируем данные из себя
        return k;
    }
};

int main() {
    auto orig = make_unique<DoorKey>();
    orig->id = "Квартира";
    auto copy = orig->clone();  // клонируем — не создаём DoorKey «с нуля»
    cout << "Оригинал и дубликат: " << orig->id << endl;  // copy тоже "Квартира"
}

Writing prototype.cpp


In [10]:
!g++ -std=c++17 -o out prototype.cpp && ./out

Оригинал и дубликат: Квартира
